# Day 2

Today, we will start using nf-core pipelines to find differentially abundant genes in our dataset. 
We are using data from the following paper: https://www.nature.com/articles/s41593-023-01350-3#Sec10

1. Please take some time to read through the paper and understand their approach, hypotheses and goals.

What was the objective of the study?

The aim of the study was to examine the transcriptomic effects of chronic exposure to oxycodone - an opioid - as well as physical dependence and addiction in presence of chronic pain with particular focussing on the brain reward circuitry.
Finding from this investigation could help to identify non-opioid medications that could ease pain during withdrawal. To do this, a mouse model was introduced in which mice first received doses of the opioid oxycodone for a certain period of time and during the next weeks it was withrdawn spontaneously. Previously these mice were divided into two main groups, one that underwent a nerve surgery resulting in neuropathic pain and one without. For both main groups, saline-treated mice were observed as control groups.

What do the conditions mean?

oxy: oxycodone (mice with or without spared nerve injury (SNI) were exposed to high doses of oxycodone for 2 weeks, and then it was withdrawn within the following 3 weeks spontaneously)


sal: saline (solution of water with salt) (control groups of mice with or without SNI were treated with not oxycodon but saline)

What do the genotypes mean?

SNI: mice with spared nerve injury


Sham: sham groups are groups that do not receive the actual treatment, but a placebo for instance and therefore can be used as control group.

Both groups have the surgical stress, but only the first groups has the SNI resulting in neuropathic pain.

Imagine you are the bioinformatician in the group who conducted this study. They hand you the raw files and ask you to analyze them.

What would you do?

Which groups would you compare to each other?

Please also mention which outcome you would expect to see from each comparison.

First, preprocessing and  a quality control and further processing of the data should be done. Afterwards, the gene expression of the different mice groups can be examined and compared doing a differential analysis.

I would focus on the comparison especially on:
 
- SNI-Oxy with Sham-Oxy  
- SNI-Oxy with SNI-Sal

With the first comparison we would be able to investigate the influence of the SNI and the resulting neuropathic pain since both groups are treated with oxycodone. I would expect that the Sham-Oxy group has only the "normal" symptomes of a withdrawl of an opioid - symptomes of physical dependance and addiction, while the mice with neuropathic pain could have changed symptomes.
With the latter comparison the influence of oxycodone and its withdrawl can be examined, because one group was treated with oxycodone and one with only saline. Here I would expect, that the withdrawl for the oxycodone treated mice could have an influence while for the ones treated with saline the withdrawl should not change much. 


Your group gave you a very suboptimal excel sheet (conditions_runs_oxy_project.xlsx) to get the information you need for each run they uploaded to the SRA.<br>
So, instead of directly diving into downloading the data and starting the analysis, you first need to sort the lazy table.<br>
Use Python and Pandas to get the table into a more sensible order.<br>
Then, perform some overview analysis and plot the results
1. How many samples do you have per condition?
2. How many samples do you have per genotype?
3. How often do you have each condition per genotype?

In [22]:
import pandas as pd
import openpyxl


# load
df = pd.read_excel('conditions_runs_oxy_project.xlsx')

# cells with NaN fill with False, cells with X replace with True
df = df.fillna(False)
df = df.replace("x", True)

# sort
df_sorted = df.sort_values(by=['condition: Sal', 'Condition: Oxy', 'Genotype: SNI', 'Genotype: Sham'])

# print for control
print(df_sorted)

# 1. How many samples do you have per condition?
print("\nNumber of samples per condition:")
print(df_sorted["Condition: Oxy"].sum())






   Patient          Run  RNA-seq  DNA-seq  condition: Sal  Condition: Oxy  \
1        ?  SRR23195506     True    False           False            True   
6        ?  SRR23195511     True    False           False            True   
9        ?  SRR23195514     True    False           False            True   
14       ?  SRR23195519     True    False           False            True   
3        ?  SRR23195508     True    False           False            True   
4        ?  SRR23195509     True    False           False            True   
11       ?  SRR23195516     True    False           False            True   
12       ?  SRR23195517     True    False           False            True   
2        ?  SRR23195507     True    False            True           False   
7        ?  SRR23195512     True    False            True           False   
10       ?  SRR23195515     True    False            True           False   
15       ?  SRR23195520     True    False            True           False   

/tmp/ipykernel_1103/1428822570.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
/tmp/ipykernel_1103/1428822570.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace("x", True)


In [23]:
# 2. How many samples do you have per genotype?

print("\nNumber of samples per genotype:")
print(df_sorted["Genotype: SNI"].sum())
print(df_sorted["Genotype: Sham"].sum())


Number of samples per genotype:
8
8


In [16]:
# 3. How often do you have each condition per genotype?

print("\nNumber of condition per genotype:")
print((df_sorted["Condition: Oxy"] & df_sorted["Genotype: SNI"]).sum())
print((df_sorted["condition: Sal"] & df_sorted["Genotype: SNI"]).sum())
print((df_sorted["Condition: Oxy"] & df_sorted["Genotype: Sham"]).sum())
print((df_sorted["condition: Sal"] & df_sorted["Genotype: Sham"]).sum())



Number of condition per genotype:
4
4
4
4


They were so kind to also provide you with the information of the number of bases per run, so that you can know how much space the data will take on your Cluster.<br>
Add a new column to your fancy table with this information (base_counts.csv) and sort your dataframe according to this information and the condition.

Then select the 2 smallest runs from your dataset and download them from SRA (maybe an nf-core pipeline can help here?...)

In [24]:
# store as variable
bases_per_run_csv = "base_counts.csv"

#load into bases
bases = pd.read_csv(bases_per_run_csv, index_col = "Run")

# print for control
bases

,Bases
Run,
SRR23195505,6922564500
SRR23195506,7859530800
SRR23195507,8063298900
SRR23195508,6927786900
SRR23195509,7003550100
SRR23195510,7377388500
SRR23195511,6456390900
SRR23195512,7462857900
SRR23195513,8099181600


In [25]:
df_sorted = df_sorted.merge(bases, on="Run")
df_sorted

,Patient,Run,RNA-seq,DNA-seq,condition: Sal,Condition: Oxy,Genotype: SNI,Genotype: Sham,Bases
0,?,SRR23195506,True,False,False,True,False,True,7859530800
1,?,SRR23195511,True,False,False,True,False,True,6456390900
2,?,SRR23195514,True,False,False,True,False,True,7226808600
3,?,SRR23195519,True,False,False,True,False,True,6996050100
4,?,SRR23195508,True,False,False,True,True,False,6927786900
5,?,SRR23195509,True,False,False,True,True,False,7003550100
6,?,SRR23195516,True,False,False,True,True,False,6203117700
7,?,SRR23195517,True,False,False,True,True,False,6863840400
8,?,SRR23195507,True,False,True,False,False,True,8063298900
9,?,SRR23195512,True,False,True,False,False,True,7462857900


In [ ]:
!nextflow run nf-core/fetchngs --input ids.csv -profile docker --max_memory "4GB" --outdir results_nf


 N E X T F L O W   ~  version 25.04.7

Launching `https://github.com/nf-core/fetchngs` [boring_ekeblad] DSL2 - revision: 8ec2d934f9 [master]

WARN: Access to undefined parameter `monochromeLogs` -- Initialise it to a default value eg. `params.monochromeLogs = some_value`


------------------------------------------------------
                                        ,--./,-.
        ___     __   __   __   ___     /,-._.--~'
  |\ | |__  __ /  ` /  \ |__) |__         }  {
  | \| |       \__, \__/ |  \ |___     \`-._,-`-,
                                        `._,._,'
  nf-core/fetchngs v1.12.0-g8ec2d93
------------------------------------------------------
Core Nextflow options
  revision       : master
  runName        : boring_ekeblad
  containerEngine: docker
  launchDir      : /mnt/c/Users/Mayal/Documents/Uni Tübingen/3. Semester/Practical - Computational workflows/computational-workflows-2025/notebooks/day_02
  workDir        : /mnt/c/Users/Mayal/Documents/Uni Tübingen/3. Semeste

While your files are downloading, get back to the paper and explain how you would try to reproduce the analysis.<br>
When you are done with this shout, so we can discuss the different ideas.

After RNA extraction and RNA-seq preparation they did a read alignment, read counting and differential analysis to in the end compare oxycodone withdrawal versus saline treatment effects in SNI and sham mice. They performed a Rank–rank hypergeometric overlap RNA-seq analysis to determine how much overlap the differential expression lists of the SNI mice with oxycodone-withdrawing and the sham group have.

(In general: Reproducability can achieved by using pipelines from nf-core.
This is very useful especially when not all information about the bioinformatical analysis is given in the paper or when it is not very detailed. In this paper the authors do not specify much and information would be missing.
Therefore it is much easier to solve this by using pipelines from nf-core, that are repdoducable, well documented and automated.)

So for reproduction, first a pipeline, such as rna_seq would be very useful for data processing and quality control, then maybe again another pipeline could be used to do the differential analysis and finally comparing the differntially expressed genes.
